---
title: 統計的学習
format: html
lang: ja
execute: 
  eval: true
ascii: false
keep-ipynb: true
jupyter:
  jupytext:
    formats: 'ipynb,qmd'
    text_representation:
      extension: .qmd
      format_name: quarto
      format_version: '1.0'
      jupytext_version: 1.14.5
  kernelspec:
    display_name: R
    language: R
    name: ir
---

In [ ]:
#| eval: false
#| echo: true
install.packages("caret")

## 教師あり学習: ペンギンデータの性別を分類するモデル

In [ ]:
#| label: ペンギンデータの読み込み
penguins <- 
  readr::read_csv("https://raw.githubusercontent.com/allisonhorst/palmerpenguins/main/inst/extdata/penguins.csv",
                  col_types = "ccddddcd")

In [ ]:
library(dplyr, warn.conflicts = FALSE)
penguins_df <- 
  penguins |> 
  # 性別が欠損値となる行を削除
  filter(!is.na(sex)) |> 
  # モデル中で利用しない変数の除外
  select(!c(year, island)) |> 
  mutate(sex = if_else(sex == "male", 1, 0))

### データの可視化

In [ ]:
library(ggplot2)

penguins_df |> 
  ggplot(aes(flipper_length_mm, bill_length_mm, 
             color = body_mass_g, 
             shape = sex,
             size = body_mass_g)) +
  geom_point(alpha = 0.5) +
  scale_color_viridis_c() +
  facet_wrap(~species)

### データ分割


データを訓練データとテストデータに分割する。

In [ ]:
set.seed(123)
train_index <- 
  sample(1:nrow(penguins_df), nrow(penguins_df) * 0.7)
train_data <- penguins_df[train_index, ] # 訓練データ（７割）
test_data <- penguins_df[-train_index, ] # テストデータ（３割）

元のデータ、訓練データ、テストデータの性別の頻度を確認する。

In [ ]:
count(penguins_df, sex)
count(train_data, sex)
count(test_data, sex)

### ロジスティック回帰

リンク関数がロジットで、誤差構造が二項分布の場合の一般化線形モデルを構築する。

訓練データを用いてモデルを学習する。

In [ ]:
fit <- glm(formula = sex ~ ., data = train_data, family = binomial)
summary(fit)

学習したモデルを用いてテストデータから性別を予測する。

In [ ]:
prob <- 
  predict(fit, newdata = test_data, type = "response")
predicted_class <- 
  if_else(prob > 0.5, 1, 0)

table(predicted_class, test_data$sex)

### 性能評価

テストデータを用いて学習したモデルの性能を評価する。

In [ ]:
# 正解率はモデルが正しく予測したデータの割合を示す
library(caret)
x <- 
  caret::confusionMatrix(factor(predicted_class), factor(test_data$sex))
x
plot(x$table, type = "heatmap", main = "Confusion Matrix")

異なるモデルを考える

In [ ]:
fit2 <- glm(formula = sex ~ body_mass_g + species, data = train_data, family = binomial)
prob2 <- 
  predict(fit2, newdata = test_data, type = "response")
predicted_class2 <- 
  if_else(prob2 > 0.5, 1, 0)
caret::confusionMatrix(factor(predicted_class2), factor(test_data$sex))

## 教師なし学習: k-平均法

In [ ]:
df_animal_tiny <-
  read.csv("https://raw.githubusercontent.com/uribo/tokupon_ds/main/data-raw/tokushima_zoo_animals22.csv") |> 
  select(body_length_cm, weight_kg) |> 
  # 欠損値を含む行を削除
  tidyr::drop_na()

In [ ]:
# データの特徴からグループ分けを行いたい
df_animal_tiny |> 
  ggplot() +
  aes(body_length_cm, weight_kg) +
  geom_point()

k-平均法で扱うデータは標準化しておく。

In [ ]:
df_animal_scaled <- 
  df_animal_tiny |> 
  mutate(across(everything(), scale))

head(df_animal_scaled)

In [ ]:
set.seed(123)
fit_kmeans <- kmeans(df_animal_scaled, centers = 3)
fit_kmeans$cluster

クラスタリングの結果をプロットして確認する。

In [ ]:
df_animal_scaled$.pred_cluster <- as.factor(fit_kmeans$cluster)

df_animal_scaled |>
  ggplot() +
  aes(x = body_length_cm, y = weight_kg, colour = .pred_cluster) +
  geom_point()